In [11]:
# Python Script to Extract tweets of a 
# particular Hashtag using Tweepy and Pandas 


# import modules 
import pandas as pd 
import tweepy 
from PIL import Image
import urllib.request as urllib2

tweet_image=None
# function to display data of each tweet 
def printtweetdata(n, ith_tweet): 
	global tweet_image
	print() 
	print(f"Tweet {n}:") 
	print(f"Username:{ith_tweet[0]}") 
	print(f"Description:{ith_tweet[1]}") 
	print(f"Location:{ith_tweet[2]}") 
	print(f"Following Count:{ith_tweet[3]}") 
	print(f"Follower Count:{ith_tweet[4]}") 
	print(f"Total Tweets:{ith_tweet[5]}") 
	print(f"Retweet Count:{ith_tweet[6]}") 
	print(f"Tweet Text:{ith_tweet[7]}") 
	print(f"Hashtags Used:{ith_tweet[8]}") 
	print(f"Tweet ID:{ith_tweet[9]}") 
	print(f"Replies:{ith_tweet[11]}")
	
	images = ith_tweet[10]
	for image in images:
		media_url=image['media_url']
		print("Images: ", media_url)
		tweet_image = Image.open(urllib2.urlopen(media_url))
		#tweet_image.show()



# function to perform data extraction 
def scrape(words, date_since, numtweet): 
	global replies
	# Creating DataFrame using pandas 
	db = pd.DataFrame(columns=['username', 'description', 'location', 'following', 
							'followers', 'totaltweets', 'retweetcount', 'text', 'hashtags','tweetID','images','replies',]) 
	
	# We are using .Cursor() to search through twitter for the required tweets. 
	# The number of tweets can be restricted using .items(number of tweets) 
	tweets = tweepy.Cursor(api.search, q=words, lang="en", 
						since='2021-01-01', 
						tweet_mode='extended',
						include_entities=True).items(numtweet) 
	
	# .Cursor() returns an iterable object. Each item in 
	# the iterator has various attributes that you can access to 
	# get information about each tweet 
	list_tweets = [tweet for tweet in tweets] 
	
	# Counter to maintain Tweet Count 
	i = 1
	
	# we will iterate over each tweet in the list for extracting information about each tweet 
	for tweet in list_tweets: 
		
		reply_tweets = tweepy.Cursor(api.search, q='to:{}'.format(user_name),
                                since_id=tweet.id, tweet_mode='extended').items()

		username = tweet.user.screen_name 
		description = tweet.user.description 
		location = tweet.user.location 
		following = tweet.user.friends_count 
		followers = tweet.user.followers_count 
		totaltweets = tweet.user.statuses_count 
		retweetcount = tweet.retweet_count 
		hashtags = tweet.entities['hashtags'] 
		images=tweet.entities['media']
		tweetId= tweet.id
		replies=[]
		while True:
			try:
				reply = reply_tweets.next()
				if not hasattr(reply, 'in_reply_to_status_id_str'):
					continue
				if reply.in_reply_to_status_id == tweetId:
					replies.append(reply.full_text)
					print("reply of tweet:{}".format(reply.full_text))

			except tweepy.RateLimitError as e:
				logging.error("Twitter api rate limit reached".format(e))
				time.sleep(60)
				continue

			except tweepy.TweepError as e:
				print("Tweepy error occured:{}".format(e))
				break

			except StopIteration:
				break

			except Exception as e:
				print("Failed while fetching replies {}".format(e))
				break
		

		# Retweets can be distinguished by a retweeted_status attribute, 
		# in case it is an invalid reference, except block will be executed 
		try: 
			text = tweet.retweeted_status.full_text 
		except AttributeError: 
			text = tweet.full_text 
		hashtext = list() 
		for j in range(0, len(hashtags)): 
			hashtext.append(hashtags[j]['text']) 
		

		# Here we are appending all the extracted information in the DataFrame 
		ith_tweet = [username, description, location, following, 
					followers, totaltweets, retweetcount, text, hashtext, tweetId, images,replies] 
		db.loc[len(db)] = ith_tweet 
		
		# Function call to print tweet data on screen 
		printtweetdata(i, ith_tweet) 

		i = i+1
	
	
	#Retweet solution:
	#api.update_status('The solution to the problem is: ', tweetId)

	#filename = 'scraped_tweets.csv'

	# we will save our database as a CSV file. 
	#db.to_csv(filename) 



# Enter your own credentials obtained 
# from your developer account 
consumer_key = "runsra61MhrLsjwgOYlANBzu9"
consumer_secret = "fVChsb8y4Aq9zGqgYrINmza8klJobvkWhZdkXhZxhjxGGa4OMZ"
access_key = "1364862750518730752-H5jTeNtJBbVxEg9iYpbZ3Ga7gZw5Oc"
access_secret = "aOQS9MUKEJ6kpLTk5Na1XSdNdcAAuq0rdalF7Hz0lIaoa"
auth = tweepy.OAuthHandler(consumer_key, consumer_secret) 
auth.set_access_token(access_key, access_secret) 
api = tweepy.API(auth)
user_name =  'xuxedu'

# Enter Hashtag and initial date 
words = "#save_my_plant" #Must include the pound(#) symbol to explicitly serach for only hashtags.
print("Searching Twitter for tweets with HashTag: ", words) 
date_since = '2021-01-01' #input() 
print("Searching for Date since(The Tweets are required in yyyy-mm--dd)") 
replies=[]

# number of tweets you want to extract in one run 
numtweet = 5
scrape(words, date_since, numtweet) 
print('Scraping has completed!') 


Searching Twitter for tweets with HashTag:  #save_my_plant
Searching for Date since(The Tweets are required in yyyy-mm--dd)
reply of tweet:Provide more  water.
reply of tweet:Maybe try adding more potassium fertilizer.

Tweet 1:
Username:xuxedu
Description:
Location:
Following Count:0
Follower Count:0
Total Tweets:6
Retweet Count:0
Tweet Text:My plant is affected by some disease, please provide me with some solution. #save_my_plant https://t.co/2UpAoVAcz4
Hashtags Used:['save_my_plant']
Tweet ID:1370295935947788295
Replies:['Provide more  water.', 'Maybe try adding more potassium fertilizer.']
Images:  http://pbs.twimg.com/media/EwRE4g4VoAErSPD.jpg
Scraping has completed!


In [23]:
import numpy as np
import pickle
from tensorflow import keras
import cv2
from os import listdir
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None :
            image = cv2.resize(image, default_image_size)   
            return img_to_array(image)
        else :
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None


def inference(image):
  model = keras.models.load_model('my_h5_model.h5')
  classes=['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy',
 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy',
 'Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_Late_blight',
 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot',
 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Target_Spot',
 'Tomato__Tomato_YellowLeaf__Curl_Virus' ,'Tomato__Tomato_mosaic_virus',
 'Tomato_healthy']
  img=image.reshape(1,256,256,3)
  value=model.predict(img)
  ind=np.argmax(value)
  print(value)
  print(classes[ind])
  return classes[ind]
  

print(keras.__version__)
import tensorflow as tf
print(tf.__version__)

#tweet_image.show()
open_cv_image = np.array(tweet_image) 
# Convert RGB to BGR 
open_cv_image = open_cv_image[:, :, ::-1].copy() 
disease_name=inference(open_cv_image)


2.4.0
2.4.0
[[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
Tomato_Early_blight


In [24]:
import mysql.connector

#Free server registered at https://www.freesqldatabase.com/account/
mydb = mysql.connector.connect(
  host="sql6.freesqldatabase.com",
  user="sql6398512",
  password="WsJANQDqVw",
  database="sql6398512"
)

print(mydb)
mycursor = mydb.cursor()

#mycursor.execute("CREATE DATABASE TrustedDB")

mycursor.execute("SHOW DATABASES")
print('Showing all databases: ')
for x in mycursor:
  print(x)

#CREATE TABLE
#mycursor.execute("CREATE TABLE trustedDBTable (disease VARCHAR(255), solutions VARCHAR(255))")

#DROP TABLE IF EXISTS
#sql = "DROP TABLE IF EXISTS trustedDBTable"
#mycursor.execute(sql)

mycursor.execute("SHOW TABLES")
print("Showing all tables:")
for x in mycursor:
  print(x)

#INSERT DISEASE NAME AND SOLUTION
sql = "INSERT INTO trustedDBTable (disease, solutions) VALUES (%s, %s)"
val = [("Tomato_Early_blight", "Add Water"),
       ("Tomato_Early_blight", "Add Potassium fertilizer")]

#mycursor.execute(sql, val)
#mycursor.executemany(sql, val)
#mydb.commit()
#print(mycursor.rowcount, "record inserted.")

#QUERY FOR SOLUTIONS

sql = "SELECT * FROM trustedDBTable WHERE disease ='"+ disease_name +"'"
mycursor.execute(sql)
myresult = mycursor.fetchall()

sql_query=[]
for x in myresult:
  print(x)
  sql_query.append(x[1])

print(sql_query)



Showing all databases: 
('information_schema',)
('sql6398512',)
Showing all tables:
('trustedDBTable',)
('Tomato_Early_blight', 'Add Water')
('Tomato_Early_blight', 'Add Potassium fertilizer')
['Add Water', 'Add Potassium fertilizer']


In [50]:

#DELETE SOLUTIONS
sql = "DELETE FROM trustedDBTable WHERE disease = 'Dry Leaf' OR solutions = 'Add fertilizer'"
mycursor.execute(sql)
mydb.commit()
print(mycursor.rowcount, "record(s) deleted")


2 record(s) deleted


In [57]:
#https://stackoverflow.com/questions/53503049/measure-similarity-between-two-documents-using-doc2vec

#More information aboout Doc2Vec: https://radimrehurek.com/gensim/models/doc2vec.html   
# https://radimrehurek.com/gensim/auto_examples/tutorials/run_doc2vec_lee.html


#Code repo on Doc2Vec https://github.com/jhlau/doc2vec, model doc2vec.bin downloaded from here.

from gensim.models import doc2vec
from scipy import spatial
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
#from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import STOPWORDS

all_stopwords_gensim = STOPWORDS
sw_list = {"not"}
selected_stopwords_gensim = STOPWORDS.difference(sw_list)


#models=['doc2vec.bin', 'GoogleNews-vectors-negative300.bin']
#d2v_model = doc2vec.Doc2Vec.load(models[0])


#Model downloaded from:
#Google news Vecor: https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit

from gensim import models
import gensim.downloader as api

import pandas as pd
import numpy as np

model = api.load('word2vec-google-news-300') #https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html#introducing-the-word2vec-model
print(model)  # output: /home/user/gensim-data/20-newsgroups/20-newsgroups.gz

# Normalizes the vectors in the word2vec class.
model.init_sims(replace=True)  

#wv = models.KeyedVectors.load_word2vec_format(wv, binary=True)



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Aswath\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [63]:

user_soln = ['pour water', 'add fertilizer', 'should give more water']
trusted_db = ['provide adequate water','add potassium fertilizer']

user_soln=replies
trusted_db=sql_query

def sanitize(text):
    #text = remove_stopwords(text)
    text_tokens = word_tokenize(text)
    tokens_without_sw = [word for word in text_tokens if not word in selected_stopwords_gensim]
    text = (" ").join(tokens_without_sw)
    text=text.strip()
    text=text.lower()
    text=text.translate(string.punctuation)
    text=text.translate(str.maketrans('', '', string.punctuation))
    print(text) 
    return text

arr=[]
for l in user_soln:
    arr.append(sanitize(l))
user_soln=arr

arr=[]
for l in trusted_db:
    arr.append(sanitize(l))
trusted_db=arr

min_dist=999

def rankit(l1,l2):
    global recommended_solution
    matrix=[]
    i=0
    for s1 in l1:
        matrix.append([])
        for s2 in l2:
            #calculate distance between two sentences using WMD algorithm
            wmdist=model.wmdistance(s1, s2)
            matrix[i].append(wmdist)
            if wmdist<min_dist:
                recommended_solution=s1
        i+=1

    df2 = pd.DataFrame(np.array(matrix), columns=l2, index=l1)
    print(df2,"\n")

#Auto correlation with user's solutions
rankit(user_soln,user_soln)

recommended_solution=""

#Cross-correlation with users and trusted solutions
rankit(user_soln,trusted_db)


print("Best solution: ",recommended_solution)


provide water 
maybe try adding potassium fertilizer 
add water
add potassium fertilizer
                                        provide water   \
provide water                                 0.000000   
maybe try adding potassium fertilizer         0.511056   

                                        maybe try adding potassium fertilizer   
provide water                                                         0.511056  
maybe try adding potassium fertilizer                                 0.000000   

                                        add water  add potassium fertilizer
provide water                            0.386386                  0.461820
maybe try adding potassium fertilizer    0.638554                  0.183589 

Best solution:  maybe try adding potassium fertilizer 


In [5]:
first_text = 'Pour some water'
second_tdifference'

def sanitize(text):
    #text = remove_stopwords(text)
    text_tokens = word_tokenize(text)
    tokens_without_sw = [word for word in text_tokens if not word in selected_stopwords_gensim]
    text = (" ").join(tokens_without_sw)
    text=text.strip()
    text=text.lower()
    text=text.translate(string.punctuation)
    text=text.translate(str.maketrans('', '', string.punctuation))
    print(text)
    return text

first_text=sanitize(first_text)
second_text=sanitize(second_text)

vec1 = d2v_model.infer_vector(first_text.split())
vec2 = d2v_model.infer_vector(second_text.split())

cos_distance = spatial.distance.cosine(vec1, vec2)
print('cosine distance',cos_distance)

distance = d2v_model.wmdistance(first_text, second_text)

print ('WM distance = %.3f' % distance)

pour water
add fertilizer
cosine distance 0.4074270725250244
WM distance = 1.272
ipykernel_launcher:25: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).


In [ ]:
tursted_db_solutions = []
user_solutions = []
for ts in tursted_db_solutions:
    for us in user_solutions:
        


In [ ]:
import gensim.downloader as api

info = api.info()  # show info about available models/datasets
model = api.load("glove-twitter-25")  # download the model and return as object ready for use
model.most_similar("cat")

In [ ]:
from keras.preprocessing import text
norm_bible="Hello world "
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(norm_bible)

word2id = tokenizer.word_index
id2word = {v:k for k, v in word2id.items()}

vocab_size = len(word2id) + 1 
embed_size = 100

wids = [[word2id[w] for w in text.text_to_word_sequence(doc)] for doc in norm_bible]
print('Vocabulary Size:', vocab_size)
print('Vocabulary Sample:', list(word2id.items())[:10])




In [3]:


from keras.preprocessing.sequence import skipgrams

# generate skip-grams
skip_grams = [skipgrams(wid, vocabulary_size=vocab_size, window_size=10) for wid in wids]

# view sample skip-grams
pairs, labels = skip_grams[0][0], skip_grams[0][1]
for i in range(10):
    print("({:s} ({:d}), {:s} ({:d})) -> {:d}".format(
          id2word[pairs[i][0]], pairs[i][0], 
          id2word[pairs[i][1]], pairs[i][1], 
          labels[i]))

Using TensorFlow backend.


OSError: [WinError 126] The specified module could not be found

In [2]:
#TODO: https://towardsdatascience.com/detecting-document-similarity-with-doc2vec-f8289a9a7db7
